# pyproj and Coordinate/Spatial Reference Systems

A Coordinate Reference System (CRS) is used to specify the location of an object on the surface of the earth through the use of coordinates.

Coordinate reference systems can be classified into two:
- **Geographic Coordinate Reference Systems** use degrees of latitude and longitude as coordinates to refer to position.
- **Projected Coordinate Reference Systems** use linear units (e.g. meters, feet, kilometers) of eastings and northings as coordinates.
T
here are other coordinate reference systems out there but these are the “classic” or “conventional” ones. Can you name some unconventional coordinate or geocode systems?

Knowledge of coordinate reference systems is important because even if two maps (or layers) show the same area, the coordinates of the locations in those maps will be different if the CRS they use are different.

**Take this example**: Map A and Map B show the same area and extent. They’re basically the same map. The only difference is the coordinate reference system they use. Map A uses CRS X and Map B uses CRS Y. Let’s say we get the coordinate of Point 1, which is (10, 10), from Map A. If we look at the coordinate (10,10) in Map B, it’s possible that Point 1 won’t be there since Map B uses a different CRS. Or if you overlay the maps over each other using a common reference, the features on the two maps won’t coincide. This knowledge of coordinate reference systems is important in any GIS.

You may notice that some Coordinate Reference Systems are referred to by their EPSG Code. This code refers to the CRS’ code in the EPSG Geodetic Parameter Dataset which is a registry of geodetic datums, spatial reference systems, Earth ellipsoids, coordinate transformations and related units of measurement. Most GIS, including QGIS, refer to the EPSG code to identify coordinate reference systems, projections, and perform transformations between these systems.

Common EPSG Codes include:
- EPSG:4326 - WGS 84, latitude/longitude coordinate system based used by the Global Positioning System (GPS) among others.
- EPSG:3857 - Web Mercator projection used for display by many web-based mapping tools such as OpenStreetMap and Google Maps
- EPSG:32650 to EPSG 32653 - Universal Transverse Mercator (UTM) Zone 50N to 53N. UTM zones used in the Philippines
- EPSG:4683 - Philippine Reference System of 1992

At it's simplest, a CRS or SRS is composed of a:
- **datum** - a model that approximates the shape of the Earth (usually a sphere or ellipsoid)
- **map projection** - a set of mathematical functions that translate locations in the surface of the datum into the Cartesian plane




## Libraries for handling reference systems
### PROJ
- https://proj.org
- open source library that implements cartographic projects and geodetic datums (see: https://proj.org/operations/projections/index.html)
- [pyproj](https://pyproj4.github.io/pyproj/stable/) is the Python inteface to proj

### GDAL/OGR
- https://gdal.org/
- The GDAL Python API also has some CRS functions via the [osgeo.osr](https://gdal.org/api/python/osgeo.osr.html) submodule

## Creating a new CRS in pyproj

pyproj implements a [`CRS`](https://pyproj4.github.io/pyproj/dev/api/crs/crs.html) class that can be instantiated in different ways.

If you want to find different CRS definitions, you can check out: [epsg.io](https://epsg.io)

For example: PRS92 (https://epsg.io/3124)

### From PROJ4 string

In [ ]:
from pyproj import CRS
prs92 = CRS.from_proj4("+proj=tmerc +lat_0=0 +lon_0=123 +k=0.99995 +x_0=500000 +y_0=0 +ellps=clrk66 +towgs84=-127.62,-67.24,-47.04,3.068,-4.903,-1.578,-1.06 +units=m +no_defs +type=crs")
prs92

### From [OGC WKT](https://www.ogc.org/standard/wkt-crs/) definition



In [ ]:
from pyproj import CRS

prs92_ogc_wkt = '''
    PROJCS["PRS92 / Philippines zone 4",
        GEOGCS["PRS92",
            DATUM["Philippine_Reference_System_1992",
                SPHEROID["Clarke 1866",6378206.4,294.978698213898],
                TOWGS84[-127.62,-67.24,-47.04,3.068,-4.903,-1.578,-1.06]],
            PRIMEM["Greenwich",0,
                AUTHORITY["EPSG","8901"]],
            UNIT["degree",0.0174532925199433,
                AUTHORITY["EPSG","9122"]],
            AUTHORITY["EPSG","4683"]],
        PROJECTION["Transverse_Mercator"],
        PARAMETER["latitude_of_origin",0],
        PARAMETER["central_meridian",123],
        PARAMETER["scale_factor",0.99995],
        PARAMETER["false_easting",500000],
        PARAMETER["false_northing",0],
        UNIT["metre",1,
            AUTHORITY["EPSG","9001"]],
        AXIS["Easting",EAST],
        AXIS["Northing",NORTH],
        AUTHORITY["EPSG","3124"]]'''

prs92 = CRS.from_wkt(prs92_ogc_wkt)
prs92

### From EPSG code

In [ ]:
from pyproj import CRS
prs92_zone3 = CRS.from_epsg(3123)
prs92_zone3

### Coordinate transformations

In [ ]:
from pyproj import CRS, Transformer

wgs84 = CRS.from_epsg(4326)
phl_prs92_zone3 = CRS.from_epsg(3123)

transformToPRS92_Zone3 = Transformer.from_crs(wgs84, phl_prs92_zone3)

transformToPRS92_Zone3.transform(14.0, 121.0) # (lat, lon) !important

## Example

In [ ]:
import fiona
ncr = fiona.open('data/phl.gpkg', layer="ncr")
ncr.crs_wkt

In [ ]:
from matplotlib import pyplot as plt
from shapely.geometry import shape

feature = next(iter(ncr))
boundaries = shape(feature["geometry"])

x,y = boundaries.geoms[0].exterior.xy

fig = plt.figure(1, dpi=120)
# Without equal aspect the map can be distorted
ax = fig.add_subplot(111, aspect='equal') 
ax.plot(x, y, color='#2f9e44', alpha=0.5,
    linewidth=1, solid_capstyle='round', zorder=2)
ax.set_title('NCR - WGS84')

Instead of WGS84, let's use PRS 92 Zone 3.

In [ ]:
transformed_coords = transformToPRS92_Zone3.transform(y, x)  # y = latitude

fig = plt.figure(1, dpi=120)
ax = fig.add_subplot(111, aspect='equal')
ax.plot(transformed_coords[0], transformed_coords[1], color='#2f9e44', alpha=0.5,
    linewidth=1, solid_capstyle='round', zorder=2)   
ax.set_title('NCR - PRS 92 Zone 3')